# Part 2- Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - We will need a Mapbox API token for some of the options:
        - https://studio.mapbox.com/
    

# Loading Data from Part 1

In [1]:
## Plotly is not included in your dojo-env
# !pip install plotly

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [3]:
## Load in csv.gz
df = pd.read_csv("Data/Baltimore-Burgers.csv.gz")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527 entries, 0 to 526
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             527 non-null    object 
 1   alias          527 non-null    object 
 2   name           527 non-null    object 
 3   image_url      508 non-null    object 
 4   is_closed      527 non-null    bool   
 5   url            527 non-null    object 
 6   review_count   527 non-null    int64  
 7   categories     527 non-null    object 
 8   rating         527 non-null    float64
 9   coordinates    527 non-null    object 
 10  transactions   527 non-null    object 
 11  price          401 non-null    object 
 12  location       527 non-null    object 
 13  phone          497 non-null    float64
 14  display_phone  497 non-null    object 
 15  distance       527 non-null    float64
dtypes: bool(1), float64(3), int64(1), object(11)
memory usage: 62.4+ KB


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,A3of5541V5SpJY-XC9WvFw,clark-burger-baltimore-3,Clark Burger,https://s3-media2.fl.yelpcdn.com/bphoto/pTrgSb...,False,https://www.yelp.com/biz/clark-burger-baltimor...,206,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 39.2854812810579, 'longitude': -7...","['delivery', 'pickup']",$$,"{'address1': '415 S Central Ave', 'address2': ...",1.667309e+10,(667) 309-3516,1404.572037
1,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,916,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"{'latitude': 39.2771835822848, 'longitude': -7...","['delivery', 'pickup']",$$,"{'address1': '1041 Marshall St', 'address2': '...",1.443454e+10,(443) 453-9698,2161.212353
2,3j6GTI0V2Jshw8e1c3uFoA,the-urban-burger-bar-baltimore,The Urban Burger Bar,https://s3-media4.fl.yelpcdn.com/bphoto/1zNJ5c...,False,https://www.yelp.com/biz/the-urban-burger-bar-...,35,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 39.32696, 'longitude': -76.63753}","['delivery', 'pickup']",$$,"{'address1': '3300 Clipper Mill Rd', 'address2...",NaN,NaN,4242.702639
3,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,136,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 39.27174, 'longitude': -76.60223}","['delivery', 'pickup']",$$,"{'address1': '823 E Fort Ave', 'address2': '',...",1.410637e+10,(410) 637-3699,2754.995467
4,DnhKT0A9OIgFdIhzNryJBw,abbey-burger-bistro-baltimore-6,Abbey Burger Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/oq4-Iy...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,268,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 39.2822776123555, 'longitude': -7...","['delivery', 'pickup']",$$,"{'address1': '811 S Broadway', 'address2': '',...",1.410522e+10,(410) 522-1428,2009.275703


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- We also want to be able to show the restaurants:
    - name,
    - price range
    - address
    - and if they do delivery or takeout.

### Separating Latitude and Longitude

In [4]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 39.2854812810579, 'longitude': -7..."
1,"{'latitude': 39.2771835822848, 'longitude': -7..."
2,"{'latitude': 39.32696, 'longitude': -76.63753}"
3,"{'latitude': 39.27174, 'longitude': -76.60223}"
4,"{'latitude': 39.2822776123555, 'longitude': -7..."
...,...
522,"{'latitude': 39.2828178405762, 'longitude': -7..."
523,"{'latitude': 39.33086, 'longitude': -76.63161}"
524,"{'latitude': 39.2821578979492, 'longitude': -7..."
525,"{'latitude': 39.322087392211, 'longitude': -76..."


- Why didn't that work???

In [5]:
## slice out a single test coordinate
test_coord = df.loc[0, 'coordinates']
test_coord

"{'latitude': 39.2854812810579, 'longitude': -76.5989856374596}"

In [6]:
type(test_coord)

str

- Its not a dictionary anymore!!! WTF??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our string dictionary into an actual dictionary. 

In [7]:
## Use json.loads on the test coordinate
# json.loads(test_coord)

- JSON requires double quotes!

In [8]:
## replace single ' with " 
test_coord = test_coord.replace("'",'"')
test_coord

'{"latitude": 39.2854812810579, "longitude": -76.5989856374596}'

In [9]:
## Use json.loads on the test coordinate, again
json.loads(test_coord)

{'latitude': 39.2854812810579, 'longitude': -76.5989856374596}

### Now, how can we apply this same process to the entire column??

In [10]:
# df['coordinates'] = df['coordinates'].str.replace("'",'"')
# df['coordinates'] = df['coordinates'].apply(json.loads)
# # df['coordinates'].apply(pd.Series)


In [11]:
lat_lon = df['coordinates'].apply(lambda cell: pd.Series(json.loads(cell.replace("'",'"'))))
lat_lon

,latitude,longitude
0,39.285481,-76.598986
1,39.277184,-76.612989
2,39.326960,-76.637530
3,39.271740,-76.602230
4,39.282278,-76.592749
...,...,...
522,39.282818,-76.582420
523,39.330860,-76.631610
524,39.282158,-76.591797
525,39.322087,-76.573110


In [12]:
## Concatenate the 2 new columns and drop the original.
df = pd.concat([df.drop(columns=['coordinates']), 
          lat_lon],axis=1)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,A3of5541V5SpJY-XC9WvFw,clark-burger-baltimore-3,Clark Burger,https://s3-media2.fl.yelpcdn.com/bphoto/pTrgSb...,False,https://www.yelp.com/biz/clark-burger-baltimor...,206,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"['delivery', 'pickup']",$$,"{'address1': '415 S Central Ave', 'address2': ...",1.667309e+10,(667) 309-3516,1404.572037,39.285481,-76.598986
1,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,916,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"['delivery', 'pickup']",$$,"{'address1': '1041 Marshall St', 'address2': '...",1.443454e+10,(443) 453-9698,2161.212353,39.277184,-76.612989
2,3j6GTI0V2Jshw8e1c3uFoA,the-urban-burger-bar-baltimore,The Urban Burger Bar,https://s3-media4.fl.yelpcdn.com/bphoto/1zNJ5c...,False,https://www.yelp.com/biz/the-urban-burger-bar-...,35,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"['delivery', 'pickup']",$$,"{'address1': '3300 Clipper Mill Rd', 'address2...",NaN,NaN,4242.702639,39.326960,-76.637530
3,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,136,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"['delivery', 'pickup']",$$,"{'address1': '823 E Fort Ave', 'address2': '',...",1.410637e+10,(410) 637-3699,2754.995467,39.271740,-76.602230
4,DnhKT0A9OIgFdIhzNryJBw,abbey-burger-bistro-baltimore-6,Abbey Burger Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/oq4-Iy...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,268,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"['delivery', 'pickup']",$$,"{'address1': '811 S Broadway', 'address2': '',...",1.410522e+10,(410) 522-1428,2009.275703,39.282278,-76.592749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,sJOcdfLvAnnBy6Vo0ylANQ,chris-seafood-baltimore,Chris' Seafood,https://s3-media1.fl.yelpcdn.com/bphoto/RpSPa-...,False,https://www.yelp.com/biz/chris-seafood-baltimo...,54,"[{'alias': 'seafood', 'title': 'Seafood'}]",3.5,"['delivery', 'pickup']",$$,"{'address1': '801 S Montford Ave', 'address2':...",1.410675e+10,(410) 675-0117,2628.043743,39.282818,-76.582420
523,rtlLrtUn35c9Y2eIcxHT8g,kings-pizza-and-subs-baltimore-2,King's Pizza & Subs,https://s3-media2.fl.yelpcdn.com/bphoto/9FEdeU...,False,https://www.yelp.com/biz/kings-pizza-and-subs-...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"['delivery', 'pickup']",$$,"{'address1': '907 W 36th St', 'address2': None...",1.410889e+10,(410) 889-3663,4382.210159,39.330860,-76.631610
524,cA-DS-nJlfl7IYre-rjH4Q,daily-grind-baltimore-8,Daily Grind,https://s3-media1.fl.yelpcdn.com/bphoto/jrsOvc...,False,https://www.yelp.com/biz/daily-grind-baltimore...,232,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,['delivery'],$,"{'address1': '1720 Thames St', 'address2': '',...",1.410558e+10,(410) 558-0399,2079.005086,39.282158,-76.591797
525,x6u_Xd5Zr6skcQsIUiUOng,jazzys-baltimore,Jazzy's,https://s3-media1.fl.yelpcdn.com/bphoto/NcDv9-...,False,https://www.yelp.com/biz/jazzys-baltimore?adju...,1,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",2.0,['delivery'],NaN,"{'address1': '3320 Belair Rd', 'address2': '',...",1.410564e+10,(410) 563-6222,4134.435594,39.322087,-76.573110


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527 entries, 0 to 526
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             527 non-null    object 
 1   alias          527 non-null    object 
 2   name           527 non-null    object 
 3   image_url      508 non-null    object 
 4   is_closed      527 non-null    bool   
 5   url            527 non-null    object 
 6   review_count   527 non-null    int64  
 7   categories     527 non-null    object 
 8   rating         527 non-null    float64
 9   transactions   527 non-null    object 
 10  price          401 non-null    object 
 11  location       527 non-null    object 
 12  phone          497 non-null    float64
 13  display_phone  497 non-null    object 
 14  distance       527 non-null    float64
 15  latitude       527 non-null    float64
 16  longitude      527 non-null    float64
dtypes: bool(1), float64(5), int64(1), object(10)
memory us

## Creating a Simple Map

### Register for MapBox API

Mapbox API: https://www.mapbox.com/

In [14]:
## Load in mapbox api credentials from .secret
with open('/Users/codingdojo/.secret/mapbox.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

- Use the plotly express `set_maptbox_acccess_token` function

In [15]:
df['longitude']

0     -76.598986
1     -76.612989
2     -76.637530
3     -76.602230
4     -76.592749
         ...    
522   -76.582420
523   -76.631610
524   -76.591797
525   -76.573110
526   -76.571850
Name: longitude, Length: 527, dtype: float64

In [16]:
# ## set mapbox token
# px.set_mapbox_access_token(login['api-key'])

In [36]:
## use scatter_mapbox for M.V.P map
px.scatter_mapbox(df,lat='latitude', lon='longitude',mapbox_style='open-street-map' )

### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - address
    - and if they do delivery or takeout.
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [37]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,A3of5541V5SpJY-XC9WvFw,clark-burger-baltimore-3,Clark Burger,https://s3-media2.fl.yelpcdn.com/bphoto/pTrgSb...,False,https://www.yelp.com/biz/clark-burger-baltimor...,206,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""415 S Central Ave"", ""address2"": ...",1.667309e+10,(667) 309-3516,1404.572037,39.285481,-76.598986
1,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,916,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1041 Marshall St"", ""address2"": ""...",1.443454e+10,(443) 453-9698,2161.212353,39.277184,-76.612989
2,3j6GTI0V2Jshw8e1c3uFoA,the-urban-burger-bar-baltimore,The Urban Burger Bar,https://s3-media4.fl.yelpcdn.com/bphoto/1zNJ5c...,False,https://www.yelp.com/biz/the-urban-burger-bar-...,35,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""3300 Clipper Mill Rd"", ""address2...",NaN,NaN,4242.702639,39.326960,-76.637530
3,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,136,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""823 E Fort Ave"", ""address2"": """",...",1.410637e+10,(410) 637-3699,2754.995467,39.271740,-76.602230
4,DnhKT0A9OIgFdIhzNryJBw,abbey-burger-bistro-baltimore-6,Abbey Burger Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/oq4-Iy...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,268,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""811 S Broadway"", ""address2"": """",...",1.410522e+10,(410) 522-1428,2009.275703,39.282278,-76.592749


In [41]:
## use scatter_mapbox for M.V.P map
px.scatter_mapbox(df,lat='latitude', lon='longitude',mapbox_style='open-street-map',
             hover_name='name',hover_data=['price','location','transactions'])

In [42]:
df['location']

0      {"address1": "415 S Central Ave", "address2": ...
1      {"address1": "1041 Marshall St", "address2": "...
2      {"address1": "3300 Clipper Mill Rd", "address2...
3      {"address1": "823 E Fort Ave", "address2": "",...
4      {"address1": "811 S Broadway", "address2": "",...
                             ...                        
522    {"address1": "801 S Montford Ave", "address2":...
523    {"address1": "907 W 36th St", "address2": None...
524    {"address1": "1720 Thames St", "address2": "",...
525    {"address1": "3320 Belair Rd", "address2": "",...
526    {"address1": "3436 Belair Rd", "address2": "",...
Name: location, Length: 527, dtype: object

In [18]:
## add hover_name (name) and hover_data for price,rating,location


### Fixing the Location Column

In [43]:
## slice out a test address
test_addr = df.loc[0, 'location']
test_addr

'{"address1": "415 S Central Ave", "address2": "", "address3": None, "city": "Baltimore", "zip_code": "21202", "country": "US", "state": "MD", "display_address": ["415 S Central Ave", "Baltimore, MD 21202"]}'

> Also a string-dictionary...

In [44]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,A3of5541V5SpJY-XC9WvFw,clark-burger-baltimore-3,Clark Burger,https://s3-media2.fl.yelpcdn.com/bphoto/pTrgSb...,False,https://www.yelp.com/biz/clark-burger-baltimor...,206,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""415 S Central Ave"", ""address2"": ...",1.667309e+10,(667) 309-3516,1404.572037,39.285481,-76.598986
1,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,916,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1041 Marshall St"", ""address2"": ""...",1.443454e+10,(443) 453-9698,2161.212353,39.277184,-76.612989
2,3j6GTI0V2Jshw8e1c3uFoA,the-urban-burger-bar-baltimore,The Urban Burger Bar,https://s3-media4.fl.yelpcdn.com/bphoto/1zNJ5c...,False,https://www.yelp.com/biz/the-urban-burger-bar-...,35,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""3300 Clipper Mill Rd"", ""address2...",NaN,NaN,4242.702639,39.326960,-76.637530
3,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,136,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""823 E Fort Ave"", ""address2"": """",...",1.410637e+10,(410) 637-3699,2754.995467,39.271740,-76.602230
4,DnhKT0A9OIgFdIhzNryJBw,abbey-burger-bistro-baltimore-6,Abbey Burger Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/oq4-Iy...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,268,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""811 S Broadway"", ""address2"": """",...",1.410522e+10,(410) 522-1428,2009.275703,39.282278,-76.592749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,sJOcdfLvAnnBy6Vo0ylANQ,chris-seafood-baltimore,Chris' Seafood,https://s3-media1.fl.yelpcdn.com/bphoto/RpSPa-...,False,https://www.yelp.com/biz/chris-seafood-baltimo...,54,"[{'alias': 'seafood', 'title': 'Seafood'}]",3.5,"['delivery', 'pickup']",$$,"{""address1"": ""801 S Montford Ave"", ""address2"":...",1.410675e+10,(410) 675-0117,2628.043743,39.282818,-76.582420
523,rtlLrtUn35c9Y2eIcxHT8g,kings-pizza-and-subs-baltimore-2,King's Pizza & Subs,https://s3-media2.fl.yelpcdn.com/bphoto/9FEdeU...,False,https://www.yelp.com/biz/kings-pizza-and-subs-...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"['delivery', 'pickup']",$$,"{""address1"": ""907 W 36th St"", ""address2"": None...",1.410889e+10,(410) 889-3663,4382.210159,39.330860,-76.631610
524,cA-DS-nJlfl7IYre-rjH4Q,daily-grind-baltimore-8,Daily Grind,https://s3-media1.fl.yelpcdn.com/bphoto/jrsOvc...,False,https://www.yelp.com/biz/daily-grind-baltimore...,232,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,['delivery'],$,"{""address1"": ""1720 Thames St"", ""address2"": """",...",1.410558e+10,(410) 558-0399,2079.005086,39.282158,-76.591797
525,x6u_Xd5Zr6skcQsIUiUOng,jazzys-baltimore,Jazzy's,https://s3-media1.fl.yelpcdn.com/bphoto/NcDv9-...,False,https://www.yelp.com/biz/jazzys-baltimore?adju...,1,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",2.0,['delivery'],NaN,"{""address1"": ""3320 Belair Rd"", ""address2"": """",...",1.410564e+10,(410) 563-6222,4134.435594,39.322087,-76.573110


In [45]:
## apply json.loads
df['location'].apply(json.loads)

JSONDecodeError: Expecting value: line 1 column 63 (char 62)

In [ ]:
df['locat']

> Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [46]:
## slice out test address 
test_addr = df.loc[0, 'location']
test_addr

'{"address1": "415 S Central Ave", "address2": "", "address3": None, "city": "Baltimore", "zip_code": "21202", "country": "US", "state": "MD", "display_address": ["415 S Central Ave", "Baltimore, MD 21202"]}'

In [51]:
result = test_addr.split('address2')
result[0]

'{"address1": "415 S Central Ave"'

In [54]:
## write a function to just run json.loads on the address
def fix_address(addr):
    try:
        return json.loads(addr)
    except:
        return "ERROR"

In [55]:
df['location'].apply(fix_address)

0                                                  ERROR
1      {'address1': '1041 Marshall St', 'address2': '...
2                                                  ERROR
3      {'address1': '823 E Fort Ave', 'address2': '',...
4      {'address1': '811 S Broadway', 'address2': '',...
                             ...                        
522    {'address1': '801 S Montford Ave', 'address2':...
523                                                ERROR
524    {'address1': '1720 Thames St', 'address2': '',...
525    {'address1': '3320 Belair Rd', 'address2': '',...
526                                                ERROR
Name: location, Length: 527, dtype: object

- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [56]:
### save a new display_location column using our function
df['display_location'] = df['location'].apply(fix_address)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location
0,A3of5541V5SpJY-XC9WvFw,clark-burger-baltimore-3,Clark Burger,https://s3-media2.fl.yelpcdn.com/bphoto/pTrgSb...,False,https://www.yelp.com/biz/clark-burger-baltimor...,206,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""415 S Central Ave"", ""address2"": ...",1.667309e+10,(667) 309-3516,1404.572037,39.285481,-76.598986,ERROR
1,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,916,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1041 Marshall St"", ""address2"": ""...",1.443454e+10,(443) 453-9698,2161.212353,39.277184,-76.612989,"{'address1': '1041 Marshall St', 'address2': '..."
2,3j6GTI0V2Jshw8e1c3uFoA,the-urban-burger-bar-baltimore,The Urban Burger Bar,https://s3-media4.fl.yelpcdn.com/bphoto/1zNJ5c...,False,https://www.yelp.com/biz/the-urban-burger-bar-...,35,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""3300 Clipper Mill Rd"", ""address2...",NaN,NaN,4242.702639,39.326960,-76.637530,ERROR
3,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,136,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""823 E Fort Ave"", ""address2"": """",...",1.410637e+10,(410) 637-3699,2754.995467,39.271740,-76.602230,"{'address1': '823 E Fort Ave', 'address2': '',..."
4,DnhKT0A9OIgFdIhzNryJBw,abbey-burger-bistro-baltimore-6,Abbey Burger Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/oq4-Iy...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,268,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""811 S Broadway"", ""address2"": """",...",1.410522e+10,(410) 522-1428,2009.275703,39.282278,-76.592749,"{'address1': '811 S Broadway', 'address2': '',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,sJOcdfLvAnnBy6Vo0ylANQ,chris-seafood-baltimore,Chris' Seafood,https://s3-media1.fl.yelpcdn.com/bphoto/RpSPa-...,False,https://www.yelp.com/biz/chris-seafood-baltimo...,54,"[{'alias': 'seafood', 'title': 'Seafood'}]",3.5,"['delivery', 'pickup']",$$,"{""address1"": ""801 S Montford Ave"", ""address2"":...",1.410675e+10,(410) 675-0117,2628.043743,39.282818,-76.582420,"{'address1': '801 S Montford Ave', 'address2':..."
523,rtlLrtUn35c9Y2eIcxHT8g,kings-pizza-and-subs-baltimore-2,King's Pizza & Subs,https://s3-media2.fl.yelpcdn.com/bphoto/9FEdeU...,False,https://www.yelp.com/biz/kings-pizza-and-subs-...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"['delivery', 'pickup']",$$,"{""address1"": ""907 W 36th St"", ""address2"": None...",1.410889e+10,(410) 889-3663,4382.210159,39.330860,-76.631610,ERROR
524,cA-DS-nJlfl7IYre-rjH4Q,daily-grind-baltimore-8,Daily Grind,https://s3-media1.fl.yelpcdn.com/bphoto/jrsOvc...,False,https://www.yelp.com/biz/daily-grind-baltimore...,232,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,['delivery'],$,"{""address1"": ""1720 Thames St"", ""address2"": """",...",1.410558e+10,(410) 558-0399,2079.005086,39.282158,-76.591797,"{'address1': '1720 Thames St', 'address2': '',..."
525,x6u_Xd5Zr6skcQsIUiUOng,jazzys-baltimore,Jazzy's,https://s3-media1.fl.yelpcdn.com/bphoto/NcDv9-...,False,https://www.yelp.com/biz/jazzys-baltimore?adju...,1,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",2.0,['delivery'],NaN,"{""address1"": ""3320 Belair Rd"", ""address2"": """",...",1.410564e+10,(410) 563-6222,4134.435594,39.322087,-76.573110,"{'address1': '3320 Belair Rd', 'address2': '',..."


In [57]:
## filter for businesses with display_location == "ERROR"
df.loc[ df['display_location']=="ERROR"]

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location
0,A3of5541V5SpJY-XC9WvFw,clark-burger-baltimore-3,Clark Burger,https://s3-media2.fl.yelpcdn.com/bphoto/pTrgSb...,False,https://www.yelp.com/biz/clark-burger-baltimor...,206,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""415 S Central Ave"", ""address2"": ...",1.667309e+10,(667) 309-3516,1404.572037,39.285481,-76.598986,ERROR
2,3j6GTI0V2Jshw8e1c3uFoA,the-urban-burger-bar-baltimore,The Urban Burger Bar,https://s3-media4.fl.yelpcdn.com/bphoto/1zNJ5c...,False,https://www.yelp.com/biz/the-urban-burger-bar-...,35,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""3300 Clipper Mill Rd"", ""address2...",NaN,NaN,4242.702639,39.326960,-76.637530,ERROR
5,jvKhto6__tCGfs2N27oWdg,the-outpost-american-tavern-baltimore-2,The Outpost American Tavern,https://s3-media2.fl.yelpcdn.com/bphoto/4bN4vA...,False,https://www.yelp.com/biz/the-outpost-american-...,184,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""1032 Riverside Ave"", ""address2"":...",1.443389e+10,(443) 388-9113,2083.657647,39.277476,-76.609180,ERROR
6,5CM8nuqd0od68GqQ-UGicQ,cookhouse-baltimore-2,CookHouse,https://s3-media2.fl.yelpcdn.com/bphoto/N-2JWj...,False,https://www.yelp.com/biz/cookhouse-baltimore-2...,20,"[{'alias': 'newamerican', 'title': 'American (...",5.0,"['delivery', 'pickup']",NaN,"{""address1"": ""1501 Bolton St"", ""address2"": Non...",1.410226e+10,(410) 225-9964,1990.078956,39.307157,-76.625848,ERROR
7,LkCSwCV-WX2eCnv5RUM3Kw,311-west-madison-ave-craft-beer-and-wine-resta...,311 West Madison Ave Craft Beer & Wine Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/g7er6A...,False,https://www.yelp.com/biz/311-west-madison-ave-...,6,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"['delivery', 'pickup']",NaN,"{""address1"": ""311 W Madison St"", ""address2"": N...",1.443939e+10,(443) 938-9109,1141.160510,39.298216,-76.620656,ERROR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,lUB7I4-nOqUWvKXcBeU9Aw,fiyah-island-grill-baltimore,Fiyah Island Grill,https://s3-media2.fl.yelpcdn.com/bphoto/k4Y4d5...,False,https://www.yelp.com/biz/fiyah-island-grill-ba...,36,"[{'alias': 'caribbean', 'title': 'Caribbean'},...",3.5,"['delivery', 'pickup']",NaN,"{""address1"": ""2416 E Monument St"", ""address2"":...",1.410675e+10,(410) 675-4000,2130.161408,39.298978,-76.583177,ERROR
514,p8_yO34vsydnG-AgoIyIMA,canton-pizza-and-deli-baltimore,Canton Pizza & Deli,https://s3-media1.fl.yelpcdn.com/bphoto/WO2KWf...,False,https://www.yelp.com/biz/canton-pizza-and-deli...,84,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"['delivery', 'pickup']",$,"{""address1"": ""3300 O""Donnell St"", ""address2"": ...",1.410564e+10,(410) 563-7777,3716.916969,39.280613,-76.569435,ERROR
519,gXQL3Zmlv0aXzjhYPRt5tg,mcdonalds-baltimore-57,McDonald's,https://s3-media2.fl.yelpcdn.com/bphoto/VAqcvo...,False,https://www.yelp.com/biz/mcdonalds-baltimore-5...,6,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",1.5,['delivery'],$,"{""address1"": ""4526 Erdman Ave"", ""address2"": No...",1.410484e+10,(410) 483-9723,4193.333190,39.309368,-76.562009,ERROR
523,rtlLrtUn35c9Y2eIcxHT8g,kings-pizza-and-subs-baltimore-2,King's Pizza & Subs,https://s3-media2.fl.yelpcdn.com/bphoto/9FEdeU...,False,https://www.yelp.com/biz/kings-pizza-and-subs-...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"['delivery', 'pickup']",$$,"{""address1"": ""907 W 36th St"", ""address2"": None...",1.410889e+10,(410) 889-3663,4382.210159,39.330860,-76.631610,ERROR


In [58]:
## slice out a new test address and inspect
test_addr = df.loc[525, 'location']
test_addr

'{"address1": "3320 Belair Rd", "address2": "", "address3": "", "city": "Baltimore", "zip_code": "21213", "country": "US", "state": "MD", "display_address": ["3320 Belair Rd", "Baltimore, MD 21213"]}'

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with "'" in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [59]:
## remove any rows where display_location == 'ERROR'
df = df.loc[ df['display_location']!="ERROR"]
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location
1,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,916,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1041 Marshall St"", ""address2"": ""...",1.443454e+10,(443) 453-9698,2161.212353,39.277184,-76.612989,"{'address1': '1041 Marshall St', 'address2': '..."
3,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,136,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""823 E Fort Ave"", ""address2"": """",...",1.410637e+10,(410) 637-3699,2754.995467,39.271740,-76.602230,"{'address1': '823 E Fort Ave', 'address2': '',..."
4,DnhKT0A9OIgFdIhzNryJBw,abbey-burger-bistro-baltimore-6,Abbey Burger Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/oq4-Iy...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,268,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""811 S Broadway"", ""address2"": """",...",1.410522e+10,(410) 522-1428,2009.275703,39.282278,-76.592749,"{'address1': '811 S Broadway', 'address2': '',..."
8,L_cDwPh8kuxkEP-VXjJF-A,shake-shack-baltimore,Shake Shack,https://s3-media2.fl.yelpcdn.com/bphoto/cORtQV...,False,https://www.yelp.com/biz/shake-shack-baltimore...,520,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"['delivery', 'pickup']",$$,"{""address1"": ""400 Pratt St"", ""address2"": """", ""...",1.443973e+10,(443) 973-3629,1017.466641,39.287108,-76.609228,"{'address1': '400 Pratt St', 'address2': '', '..."
10,6PLtSs9Bi4QDSNBM_u4fNg,the-brewers-art-baltimore,The Brewer's Art,https://s3-media1.fl.yelpcdn.com/bphoto/2pkJFg...,False,https://www.yelp.com/biz/the-brewers-art-balti...,1041,"[{'alias': 'breweries', 'title': 'Breweries'},...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1106 N Charles St"", ""address2"": ...",1.410548e+10,(410) 547-6925,1040.671677,39.302825,-76.616180,"{'address1': '1106 N Charles St', 'address2': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,dWI0jCoOqjge9VQTIad-Ww,york-pizza-subs-and-wings-baltimore,York Pizza Subs and Wings,https://s3-media4.fl.yelpcdn.com/bphoto/Ay3jyi...,False,https://www.yelp.com/biz/york-pizza-subs-and-w...,51,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.0,"['delivery', 'pickup']",$,"{""address1"": ""3401 Greenmount Ave"", ""address2""...",1.410236e+10,(410) 235-6290,3743.767034,39.329853,-76.609146,"{'address1': '3401 Greenmount Ave', 'address2'..."
521,WVQqkLAKowuIe51IVNXCZA,outback-steakhouse-baltimore,Outback Steakhouse,https://s3-media3.fl.yelpcdn.com/bphoto/a8O0HB...,False,https://www.yelp.com/biz/outback-steakhouse-ba...,135,"[{'alias': 'steak', 'title': 'Steakhouses'}]",3.0,"['delivery', 'pickup']",$$,"{""address1"": ""2400 Boston St"", ""address2"": """",...",1.410523e+10,(410) 522-7757,2755.986468,39.281747,-76.581631,"{'address1': '2400 Boston St', 'address2': '',..."
522,sJOcdfLvAnnBy6Vo0ylANQ,chris-seafood-baltimore,Chris' Seafood,https://s3-media1.fl.yelpcdn.com/bphoto/RpSPa-...,False,https://www.yelp.com/biz/chris-seafood-baltimo...,54,"[{'alias': 'seafood', 'title': 'Seafood'}]",3.5,"['delivery', 'pickup']",$$,"{""address1"": ""801 S Montford Ave"", ""address2"":...",1.410675e+10,(410) 675-0117,2628.043743,39.282818,-76.582420,"{'address1': '801 S Montford Ave', 'address2':..."
524,cA-DS-nJlfl7IYre-rjH4Q,daily-grind-baltimore-8,Daily Grind,https://s3-media1.fl.yelpcdn.com/bphoto/jrsOvc...,False,https://www.yelp.com/biz/daily-grind-baltimore...,232,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,['delivery'],$,"{""address1"": ""1720 Thames St"", "

In [60]:
df.loc[525,'display_location']

{'address1': '3320 Belair Rd',
 'address2': '',
 'address3': '',
 'city': 'Baltimore',
 'zip_code': '21213',
 'country': 'US',
 'state': 'MD',
 'display_address': ['3320 Belair Rd', 'Baltimore, MD 21213']}

- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [63]:
## use apply and lambda to slice correct key
df['display_address'] = df['display_location'].apply(lambda x: x['display_address'])
df

/var/folders/rf/vw4r41jd7vd95x1w0dth7v9h0000gp/T/ipykernel_58880/1758132525.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location,display_address
1,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,916,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1041 Marshall St"", ""address2"": ""...",1.443454e+10,(443) 453-9698,2161.212353,39.277184,-76.612989,"{'address1': '1041 Marshall St', 'address2': '...","[1041 Marshall St, Baltimore, MD 21230]"
3,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,136,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""823 E Fort Ave"", ""address2"": """",...",1.410637e+10,(410) 637-3699,2754.995467,39.271740,-76.602230,"{'address1': '823 E Fort Ave', 'address2': '',...","[823 E Fort Ave, Baltimore, MD 21230]"
4,DnhKT0A9OIgFdIhzNryJBw,abbey-burger-bistro-baltimore-6,Abbey Burger Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/oq4-Iy...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,268,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""811 S Broadway"", ""address2"": """",...",1.410522e+10,(410) 522-1428,2009.275703,39.282278,-76.592749,"{'address1': '811 S Broadway', 'address2': '',...","[811 S Broadway, Baltimore, MD 21231]"
8,L_cDwPh8kuxkEP-VXjJF-A,shake-shack-baltimore,Shake Shack,https://s3-media2.fl.yelpcdn.com/bphoto/cORtQV...,False,https://www.yelp.com/biz/shake-shack-baltimore...,520,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"['delivery', 'pickup']",$$,"{""address1"": ""400 Pratt St"", ""address2"": """", ""...",1.443973e+10,(443) 973-3629,1017.466641,39.287108,-76.609228,"{'address1': '400 Pratt St', 'address2': '', '...","[400 Pratt St, Baltimore, MD 21202]"
10,6PLtSs9Bi4QDSNBM_u4fNg,the-brewers-art-baltimore,The Brewer's Art,https://s3-media1.fl.yelpcdn.com/bphoto/2pkJFg...,False,https://www.yelp.com/biz/the-brewers-art-balti...,1041,"[{'alias': 'breweries', 'title': 'Breweries'},...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1106 N Charles St"", ""address2"": ...",1.410548e+10,(410) 547-6925,1040.671677,39.302825,-76.616180,"{'address1': '1106 N Charles St', 'address2': ...","[1106 N Charles St, Baltimore, MD 21201]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,dWI0jCoOqjge9VQTIad-Ww,york-pizza-subs-and-wings-baltimore,York Pizza Subs and Wings,https://s3-media4.fl.yelpcdn.com/bphoto/Ay3jyi...,False,https://www.yelp.com/biz/york-pizza-subs-and-w...,51,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.0,"['delivery', 'pickup']",$,"{""address1"": ""3401 Greenmount Ave"", ""address2""...",1.410236e+10,(410) 235-6290,3743.767034,39.329853,-76.609146,"{'address1': '3401 Greenmount Ave', 'address2'...","[3401 Greenmount Ave, Baltimore, MD 21218]"
521,WVQqkLAKowuIe51IVNXCZA,outback-steakhouse-baltimore,Outback Steakhouse,https://s3-media3.fl.yelpcdn.com/bphoto/a8O0HB...,False,https://www.yelp.com/biz/outback-steakhouse-ba...,135,"[{'alias': 'steak', 'title': 'Steakhouses'}]",3.0,"['delivery', 'pickup']",$$,"{""address1"": ""2400 Boston St"", ""address2"": """",...",1.410523e+10,(410) 522-7757,2755.986468,39.281747,-76.581631,"{'address1': '2400 Boston St', 'address2': '',...","[2400 Boston St, Baltimore, MD 21224]"
522,sJOcdfLvAnnBy6Vo0ylANQ,chris-seafood-baltimore,Chris' Seafood,https://s3-media1.fl.yelpcdn.com/bphoto/RpSPa-...,False,https://www.yelp.com/biz/chris-seafood-baltimo...,54,"[{'alias': 'seafood', 'title': 'Seafood'}]",3.5,"['delivery', 'pickup']",$$,"{""address1"": ""801 S Montford Ave"", ""address2"":...",1.410675e+10,(410) 675-0117,2628.043743,39.282818,-76.582420,"{'address1': '801 S Mont

- Almost done! We want to convert display_address to a string instead a list of strings.
- We can use the string method .join to do so!

In [66]:
## slice out a test_address
test = df.loc[1,'display_address']
test

['1041 Marshall St', 'Baltimore, MD 21230']

In [68]:
## test using .join with a "\n"
print("\n".join(test))

1041 Marshall St
Baltimore, MD 21230


In [69]:
## apply the join to every row with a lambda
df['Address'] = df['display_address'].apply(lambda x: "\n".join(x))
df

/var/folders/rf/vw4r41jd7vd95x1w0dth7v9h0000gp/T/ipykernel_58880/3208991675.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location,display_address,Address
1,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,916,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1041 Marshall St"", ""address2"": ""...",1.443454e+10,(443) 453-9698,2161.212353,39.277184,-76.612989,"{'address1': '1041 Marshall St', 'address2': '...","[1041 Marshall St, Baltimore, MD 21230]","1041 Marshall St\nBaltimore, MD 21230"
3,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,136,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""823 E Fort Ave"", ""address2"": """",...",1.410637e+10,(410) 637-3699,2754.995467,39.271740,-76.602230,"{'address1': '823 E Fort Ave', 'address2': '',...","[823 E Fort Ave, Baltimore, MD 21230]","823 E Fort Ave\nBaltimore, MD 21230"
4,DnhKT0A9OIgFdIhzNryJBw,abbey-burger-bistro-baltimore-6,Abbey Burger Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/oq4-Iy...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,268,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""811 S Broadway"", ""address2"": """",...",1.410522e+10,(410) 522-1428,2009.275703,39.282278,-76.592749,"{'address1': '811 S Broadway', 'address2': '',...","[811 S Broadway, Baltimore, MD 21231]","811 S Broadway\nBaltimore, MD 21231"
8,L_cDwPh8kuxkEP-VXjJF-A,shake-shack-baltimore,Shake Shack,https://s3-media2.fl.yelpcdn.com/bphoto/cORtQV...,False,https://www.yelp.com/biz/shake-shack-baltimore...,520,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"['delivery', 'pickup']",$$,"{""address1"": ""400 Pratt St"", ""address2"": """", ""...",1.443973e+10,(443) 973-3629,1017.466641,39.287108,-76.609228,"{'address1': '400 Pratt St', 'address2': '', '...","[400 Pratt St, Baltimore, MD 21202]","400 Pratt St\nBaltimore, MD 21202"
10,6PLtSs9Bi4QDSNBM_u4fNg,the-brewers-art-baltimore,The Brewer's Art,https://s3-media1.fl.yelpcdn.com/bphoto/2pkJFg...,False,https://www.yelp.com/biz/the-brewers-art-balti...,1041,"[{'alias': 'breweries', 'title': 'Breweries'},...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1106 N Charles St"", ""address2"": ...",1.410548e+10,(410) 547-6925,1040.671677,39.302825,-76.616180,"{'address1': '1106 N Charles St', 'address2': ...","[1106 N Charles St, Baltimore, MD 21201]","1106 N Charles St\nBaltimore, MD 21201"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,dWI0jCoOqjge9VQTIad-Ww,york-pizza-subs-and-wings-baltimore,York Pizza Subs and Wings,https://s3-media4.fl.yelpcdn.com/bphoto/Ay3jyi...,False,https://www.yelp.com/biz/york-pizza-subs-and-w...,51,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.0,"['delivery', 'pickup']",$,"{""address1"": ""3401 Greenmount Ave"", ""address2""...",1.410236e+10,(410) 235-6290,3743.767034,39.329853,-76.609146,"{'address1': '3401 Greenmount Ave', 'address2'...","[3401 Greenmount Ave, Baltimore, MD 21218]","3401 Greenmount Ave\nBaltimore, MD 21218"
521,WVQqkLAKowuIe51IVNXCZA,outback-steakhouse-baltimore,Outback Steakhouse,https://s3-media3.fl.yelpcdn.com/bphoto/a8O0HB...,False,https://www.yelp.com/biz/outback-steakhouse-ba...,135,"[{'alias': 'steak', 'title': 'Steakhouses'}]",3.0,"['delivery', 'pickup']",$$,"{""address1"": ""2400 Boston St"", ""address2"": """",...",1.410523e+10,(410) 522-7757,2755.986468,39.281747,-76.581631,"{'address1': '2400 Boston St', 'address2': '',...","[2400 Boston St, Baltimore, MD 21224]","2400 Boston St\nBaltimore, MD 21224"
522,sJOcdfLvAnnBy6Vo0ylANQ,chris-seafood-baltimore,Chris' Seafood,https://s3-media1.fl.yelpcdn.com/bphoto/RpSP

### Final Map

In [77]:
## use scatter_mapbox for M.V.P map
fig = px.scatter_mapbox(df.fillna('NA'),lat='latitude', lon='longitude',mapbox_style='open-street-map',
             hover_name='name',hover_data=['price','Address','transactions'],
                 color='rating')
fig

#### HTML Uses `<br>` instead of `\n`

In [34]:
## remake the final address column with <br> instead 

## plot the final map

In [73]:
## use fig.write_html to save map
fig.write_html('final_map.html')

# Investigating Price

In [75]:
df['price'].value_counts(dropna=False)

$$      175
$        89
NaN      30
$$$      19
$$$$      1
Name: price, dtype: int64